In [1]:
## Import Modules
import numpy as np
from matplotlib import pyplot as plt

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam

from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
## import MNIST datasets
from keras.datasets import mnist
#(X_train2, y_train2), (X_test2, y_test2) = mnist.load_data()

b = np.load('./data/mnist.npz')
## train and test data
X_train2, y_train2, X_test2, y_test2 = b['x_train'], b['y_train'], b['x_test'], b['y_test']

In [3]:
print(X_train2.shape)
print(X_test2.shape)
print(y_train2.shape)
print(y_test2.shape)

(60000, 28, 28)
(10000, 28, 28)
(60000,)
(10000,)


In [4]:
## Preare Train and Test data
img_rows, img_cols = X_train2[0].shape[0], X_train2[0].shape[1]

## Prepare X data by reshaping to 3 dimensions
X_train3 = X_train2.reshape(X_train2.shape[0], img_rows, img_cols, 1)
X_test3 = X_test2.reshape(X_test2.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1) ## 28 by 28 by 1

In [5]:
print("Image row and columns ; ", img_rows, img_cols)
print("Shape of Train data : ", X_train3.shape)
print("Shape of Test data : ", X_test3.shape)
print("Shape of input layer : ", input_shape)

print("Number of records: ", X_train2.shape[0])

Image row and columns ;  28 28
Shape of Train data :  (60000, 28, 28, 1)
Shape of Test data :  (10000, 28, 28, 1)
Shape of input layer :  (28, 28, 1)
Number of records:  60000


In [6]:
## Normalize the data
X_train = X_train3.astype('float32')/255.
X_test = X_test3.astype('float32')/255.

In [7]:
## One hot encoding 
n_classes = len(set(y_train2))
y_train = np_utils.to_categorical(y_train2, n_classes)
y_test = np_utils.to_categorical(y_test2, n_classes)

In [8]:
print(y_train.shape)

(60000, 10)


In [9]:
## Buiid CNN model

model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss=['categorical_crossentropy'], optimizer='adam', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 6, 256)         295168    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 3, 3, 256)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 3, 3, 256)         0         
__________

In [10]:
## Get Callbacks
callbacks = [EarlyStopping(monitor='val_acc', patience=5)]

In [11]:
batch_size = 128
n_epochs = 10

In [12]:
## Train the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=n_epochs, verbose=1, validation_split=0.2, callbacks=callbacks)

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 120s 3ms/step - loss: 0.3117 - acc: 0.9003 - val_loss: 0.0685 - val_acc: 0.9800
Epoch 2/10
48000/48000 [==============================] - 118s 2ms/step - loss: 0.0879 - acc: 0.9734 - val_loss: 0.0382 - val_acc: 0.9889
Epoch 3/10
48000/48000 [==============================] - 118s 2ms/step - loss: 0.0633 - acc: 0.9812 - val_loss: 0.0311 - val_acc: 0.9914
Epoch 4/10
48000/48000 [==============================] - 118s 2ms/step - loss: 0.0513 - acc: 0.9846 - val_loss: 0.0288 - val_acc: 0.9915
Epoch 5/10
48000/48000 [==============================] - 117s 2ms/step - loss: 0.0420 - acc: 0.9874 - val_loss: 0.0264 - val_acc: 0.9918
Epoch 6/10
48000/48000 [==============================] - 118s 2ms/step - loss: 0.0357 - acc: 0.9892 - val_loss: 0.0282 - val_acc: 0.9917
Epoch 7/10
48000/48000 [==============================] - 118s 2ms/step - loss: 0.0344 - acc: 0.9895 - val_loss: 0.0237 - 

In [13]:
## Evaludation socre
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.01834804192893098
Test accuracy: 0.9944
